#  Домашнее задание по NLP # 3 [100 баллов] 
## Классификация и генерация фамилий 

В этом домашнем задании вам предстоит классифицировать и генерировать фамилии на 19 разных языках. Обучающие данные хранятся в папке data и разбиты по языкам: один язык – одна подпапка. Ниже представлен код для считывания данных в словарь вида: 
```d{язык} : [список имен]```.

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return lines

all_categories = []
category_lines = {}
for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

print(category_lines['Russian'][100:120])

['data/names/Japanese.txt', 'data/names/English.txt', 'data/names/Chinese.txt', 'data/names/Portuguese.txt', 'data/names/Irish.txt', 'data/names/Scottish.txt', 'data/names/Polish.txt', 'data/names/German.txt', 'data/names/Greek.txt', 'data/names/Russian.txt', 'data/names/Dutch.txt', 'data/names/Spanish.txt', 'data/names/Korean.txt', 'data/names/Vietnamese.txt', 'data/names/Arabic.txt', 'data/names/French.txt', 'data/names/Italian.txt', 'data/names/Czech.txt']
['Abels', 'Abelsky', 'Abeltsev', 'Abelyan', 'Aberson', 'Abertasov', 'Abesadze', 'Abezgauz', 'Abgaryan', 'Abibulaev', 'Abidoff', 'Abidov', 'Abih', 'Abikh', 'Abisaloff', 'Abisalov', 'Abitoff', 'Abitov', 'Abjaliloff', 'Abjalilov']


In [2]:
print(category_lines['Chinese'][100:120])

['Miao', 'Min', 'Ming', 'Moy', 'Mui', 'Nie', 'Niu', 'Ou-Yang', 'Ow-Yang', 'Pan', 'Pang', 'Pei', 'Peng', 'Ping', 'Qian', 'Qin', 'Qiu', 'Quan', 'Que', 'Ran']


## Часть 1. Предварительная обработка данных [10 баллов]

1. Удалите неоднозначные фамилии (т.е. одинаковые фамилии на разных языка), если такие есть;
2. Оцените 
* среднюю длину фамилии по всей коллекции
* по каждому языку
3. Для последующей классификации (части 2 и 3) оздайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы: то есть, в обучающее множество должно входить примерно одинаковое количество фамилий на разных ящыка. 

In [3]:
#Удаляем одинаковые фамилии из разных языков. Для этого в цикле проходим все языки и поочередно сравниваем вся фамилии 
#из первого и второго списка
category1={}
category2={}
for i in range(len(all_categories)):
    category1=set(category_lines[all_categories[i]])
    for j in range(i+1, len(all_categories)):
        category2=set(category_lines[all_categories[j]])
        category2-=category1
        category_lines[j]=list(category2)

In [4]:
#Вычисляем среднее количество букв в фамилии для каждого языка и в общем для выборки
categories=dict()
for cat in all_categories:
    category=category_lines[cat]
    l=0
    for c in category:
        l+=len(c)
    col=len(category)
    m=l/col
    categories[cat]=m
col=0
sr=0
for c in categories:
    col+=categories[c]
sr=col/len(categories)
print('Среднее количество букв в фамилии:', sr)
for category in categories:
    print(category, ';', categories[category])


Среднее количество букв в фамилии: 6.329515213451118
Japanese ; 6.691220988900101
English ; 6.395038167938932
Chinese ; 3.6529850746268657
Portuguese ; 6.418918918918919
Irish ; 7.004310344827586
Scottish ; 6.52
Polish ; 7.266187050359712
German ; 6.669889502762431
Greek ; 8.758620689655173
Russian ; 8.09906462585034
Dutch ; 6.915824915824916
Spanish ; 6.4865771812080535
Korean ; 3.5
Vietnamese ; 3.643835616438356
Arabic ; 5.595
French ; 6.805054151624549
Italian ; 6.957686882933709
Czech ; 6.551059730250482


In [5]:
#Считаем частотность букв в каждом из языков и записываем это в словарь
categoriesLet=dict()
for cat in all_categories:
    category=category_lines[cat]
    alf=dict()
    for c in category:
        alf[c[0]]=alf.get(c[0], 0)+1
    categoriesLet[cat]=alf

In [6]:
categoriesLet

{'Arabic': {'A': 276,
  'B': 194,
  'C': 25,
  'D': 59,
  'E': 13,
  'F': 28,
  'G': 134,
  'H': 162,
  'I': 31,
  'K': 176,
  'M': 197,
  'N': 143,
  'Q': 33,
  'R': 20,
  'S': 352,
  'T': 115,
  'W': 21,
  'Z': 21},
 'Chinese': {'A': 2,
  'B': 7,
  'C': 23,
  'D': 7,
  'E': 1,
  'F': 7,
  'G': 16,
  'H': 17,
  'J': 11,
  'K': 13,
  'L': 20,
  'M': 13,
  'N': 3,
  'O': 2,
  'P': 6,
  'Q': 8,
  'R': 6,
  'S': 28,
  'T': 17,
  'W': 10,
  'X': 12,
  'Y': 19,
  'Z': 20},
 'Czech': {'A': 6,
  'B': 30,
  'C': 27,
  'D': 18,
  'E': 2,
  'F': 14,
  'G': 12,
  'H': 41,
  'J': 20,
  'K': 73,
  'L': 7,
  'M': 34,
  'N': 18,
  'O': 12,
  'P': 44,
  'Q': 1,
  'R': 16,
  'S': 81,
  'T': 14,
  'U': 7,
  'V': 16,
  'W': 14,
  'Z': 12},
 'Dutch': {'A': 30,
  'B': 9,
  'C': 3,
  'D': 8,
  'H': 21,
  'K': 34,
  'L': 13,
  'M': 24,
  'N': 8,
  'O': 10,
  'P': 17,
  'R': 50,
  'S': 53,
  'T': 7,
  'V': 9,
  'Z': 1},
 'English': {'A': 199,
  'B': 200,
  'C': 200,
  'D': 200,
  'E': 186,
  'F': 200,
  'G': 

### Разбиение на обучающую и тестовую выборки

In [7]:
#Определяем размер обучающей выгрузки
size=0.8

In [8]:
#Для каждой категории определяем какое количество фамилиий на каждую букву должно попасть в обучающую выборку
for cat in categoriesLet:
    category=categoriesLet[cat]
    for c in category:
        category[c]=round(category[c]*size)
#print(categoriesLet)

In [9]:
#Создаем два словаря, в который будем заносить фамилии для обучающей и тестовой выборки
import random
train=dict()
test=dict()

for cat in all_categories:
    category=category_lines[cat]
    letters=categoriesLet[cat]
    l=len(category)
    trainf=list()
    testf=list()
    for i in range(l):
        k=random.choice(category)
        if letters.get(k[0], 0)>0:             
            trainf.append(k)
            letters[k[0]]=letters[k[0]]-1
        else:
            testf.append(k)
        category.remove(k)
    train[cat]=trainf
    test[cat]=testf

In [10]:
#Тестовая выборка
print(test)

{'Japanese': ['Chikanatsu', 'Horigoshi', 'Nonomura', 'Hirano', 'Ayugai', 'Yamanoue', 'Nakahara', 'Arita', 'Tamuro', 'Hamada', 'Hagino', 'Nakasawa', 'Goto', 'Terauchi', 'Nagai', 'Narita', 'Akamatsu', 'Hirasi', 'Yamaguchiya', 'Yamashita', 'Yamabe', 'Fukusaku', 'Hiranuma', 'Takenouchi', 'Taku', 'Watari', 'Hideyoshi', 'Fukao', 'Nijo', 'Yoshikawa', 'Masuko', 'Kate', 'Kamioka', 'Komon', 'Marusa', 'Koizumi', 'Karubo', 'Kodama', 'Shimohira', 'Toru', 'Okuma', 'Sakai', 'Furusawa', 'Mushashibo', 'Iseya', 'Shimamura', 'Ieyasu', 'Kokan', 'Shinozaki', 'Kunikida', 'Fujikage', 'Okura', 'Imoo', 'Sarumara', 'Sugitani', 'Shimada', 'Ekiguchi', 'Terada', 'Takei', 'Motoyoshi', 'Takayama', 'Kasamatsu', 'Ueda', 'Morimoto', 'Bunya', 'Ebina', 'Hokusai', 'Yamura', 'Shibasaki', 'Tsukamoto', 'Marubeni', 'Kikkawa', 'Segawa', 'Sone', 'Okazaki', 'Shidehara', 'Takemago', 'Asahi', 'Tsumura', 'Mifune', 'Kido', 'Ezakiya', 'Kawatake', 'Tsuda', 'Takara', 'Hamaguchi', 'Kimiyama', 'Takizawa', 'Kudara', 'Amaya', 'Kamon', 'Ter

In [11]:
#Обучающая выборка
print(train)

{'Japanese': ['Sada', 'Isoda', 'Takaoka', 'Shige', 'Shikuk', 'Sunada', 'Tamazaki', 'Odaka', 'Inouye', 'Sasaki', 'Miyajima', 'Yamhata', 'Komatsuzaki', 'Abe', 'Nobunaga', 'Yoshida', 'Genda', 'Eda', 'Amagawa', 'Hoshino', 'Togo', 'Tokuda', 'Yamana', 'Deguchi', 'Kawaii', 'Takudo', 'Tomioka', 'Hasegawa', 'Jimbo', 'Watnabe', 'Yakuta', 'Shirane', 'Shibaguchi', 'Matsushina', 'Mimasuya', 'Hosokawa', 'Nishimoto', 'Hiro', 'Sekine', 'Onoda', 'Minatoya', 'Matsuya', 'Kamisaka', 'Kogo', 'Miyazawa', 'Ota', 'Shikitei', 'Iida', 'Ibi', 'Chikamatsu', 'Hitomi', 'Kamata', 'Umeki', 'Okita', 'Kasaya', 'Ishinomori', 'Uetake', 'Yonai', 'Chino', 'Honda', 'Maruya', 'Kunisada', 'Higuchi', 'Yanagita', 'Matsukata', 'Arakida', 'Yasuda', 'Iwakura', 'Kobi', 'Itoh', 'Hama', 'Arishima', 'Hojo', 'Inukai', 'Yokokawa', 'Imada', 'Ohmae', 'Ono', 'Kan', 'Sakuraba', 'Satake', 'Akiyama', 'Saga', 'Sakata', 'Minamoto', 'Kanesaka', 'Fukunaka', 'Narato', 'Hirase', 'Yamahata', 'Asano', 'Yamaoka', 'Ogura', 'Nakamoto', 'Kotoku', 'Shiroy

##  Часть 2. Базовый метод классификации [20 баллов]



Используйте метод наивного Байеса, логистическую регрессию или любой другой метод для классификации фамилий: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [12]:
import nltk
from nltk.util import ngrams
from tqdm import tqdm

In [13]:
import pandas as pd
columns = ['Language', 'Family']
df = pd.DataFrame(columns=columns) 
dftest = pd.DataFrame(columns=columns) 

In [14]:
for cat in train:
    category=train[cat]
    for f in category:
        df.loc[len(df)]=[cat, f] 
df.head()

,Language,Family
0,Japanese,Sada
1,Japanese,Isoda
2,Japanese,Takaoka
3,Japanese,Shige
4,Japanese,Shikuk


In [15]:
for cat in tqdm(test):
    category=test[cat]
    for f in category:
        dftest.loc[len(dftest)]=[cat, f] 
dftest.head()

100%|██████████| 18/18 [00:10<00:00,  1.67it/s]


,Language,Family
0,Japanese,Chikanatsu
1,Japanese,Horigoshi
2,Japanese,Nonomura
3,Japanese,Hirano
4,Japanese,Ayugai


### N=2

Разбиваем слова на n-граммы, записываем все в словарь, а так же создаем список n-грамм

In [16]:
n=2

In [17]:
train_ng=dict()
names=set()
for cat in tqdm(train):
    category=train[cat]
    cat_ng=dict()
    for f in category:
        ngs=[ng for ng in nltk.ngrams(f, n)]
        cat_ng[f]=ngs
        for ng in ngs:
            names.add(ng)
    train_ng[cat]=cat_ng

100%|██████████| 18/18 [00:00<00:00, 109.40it/s]


In [18]:
len(train_ng)

18

In [19]:
test_ng=dict()
for cat in test:
    category=test[cat]
    cat_ng=dict()
    for f in category:
        ngs=[ng for ng in nltk.ngrams(f, n)]
        cat_ng[f]=ngs
        for ng in ngs:
            names.add(ng)
    test_ng[cat]=cat_ng

In [20]:
#Заполняем нулями обучающую матрицу
for name in names:
    df[name]=df.fillna(0, inplace=True)

In [21]:
#Заполняем нулями тестовую матрицу
for name in tqdm(names):
    dftest[name]=dftest.fillna(0, inplace=True)

100%|██████████| 1166/1166 [00:19<00:00, 59.77it/s]


In [22]:
%time
#Заполняем 1 поля на пересечении фамилии и входящей в ее состав n-граммы
for cat in tqdm(all_categories):
    family = train_ng[cat]
    for f in family:
        ngs=family[f]
        for ng in ngs:
            df.loc[(df['Language'] == cat) & (df['Family'] == family), ng] = 1

  0%|          | 0/18 [00:00<?, ?it/s]

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.63 µs


100%|██████████| 18/18 [12:44<00:00, 42.49s/it]


In [23]:
df.head()

,Language,Family,"(K, u)","(j, z)","(H, é)","(p, r)","(d, i)","(í, v)","(i, è)","(I, e)",...,"(e, h)","(s, i)","(', K)","(g, h)","(ß, e)","(e, j)","(í, g)","(d, à)","(m, u)","(s, a)"
0,Japanese,Sada,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
1,Japanese,Isoda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
2,Japanese,Takaoka,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
3,Japanese,Shige,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
4,Japanese,Shikuk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None


In [24]:
#Заполняем 1 поля на пересечении фамилии и входящей в ее состав n-граммы
for cat in tqdm(all_categories):
    family = test_ng[cat]
    for f in family:
        ngs=family[f]
        for ng in ngs:
            dftest.loc[(dftest['Language'] == cat) & (dftest['Family'] == family), ng] = 1

100%|██████████| 18/18 [02:14<00:00,  7.47s/it]


In [25]:
Y_train=df['Language']
Y_test=dftest['Language']

In [26]:
dftest.head()

,Language,Family,"(K, u)","(j, z)","(H, é)","(p, r)","(d, i)","(í, v)","(i, è)","(I, e)",...,"(e, h)","(s, i)","(', K)","(g, h)","(ß, e)","(e, j)","(í, g)","(d, à)","(m, u)","(s, a)"
0,Japanese,Chikanatsu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
1,Japanese,Horigoshi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
2,Japanese,Nonomura,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
3,Japanese,Hirano,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
4,Japanese,Ayugai,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None


In [27]:
df.drop('Language', axis=1, inplace=True)
dftest.drop('Language', axis=1, inplace=True)
df.drop('Family', axis=1, inplace=True)
dftest.drop('Family', axis=1, inplace=True)

In [30]:
df.drop(('s', 'a'), axis=1, inplace=True)
dftest.drop(('s', 'a'), axis=1, inplace=True)

### Logistic Regression

In [31]:
from sklearn.metrics import auc, roc_curve
from sklearn.linear_model import LogisticRegression

In [32]:
clf = LogisticRegression(C=1)
clf.fit(df, Y_train) 
y_pred_l = clf.predict(df)
y_t_pred_l = clf.predict(dftest)

In [34]:
from sklearn.metrics import *

In [35]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_train, y_pred_l, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_train, y_pred_l)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_test, y_t_pred_l, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_test, y_t_pred_l)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Random Forest

In [37]:
from sklearn.metrics import auc, roc_curve
from sklearn.linear_model import LogisticRegression

In [38]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=200, max_depth=15, min_samples_leaf=20, max_features=0.8, n_jobs=-1)
clf_rf.fit(df, Y_train)    

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features=0.8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
y_pred = clf_rf.predict(df)
y_t_pred = clf_rf.predict(dftest)

In [40]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [41]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_train, y_pred, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_train, y_pred)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_test, y_t_pred, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_test, y_t_pred)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## N=3

Заводим новые датафреймы для обучения и для теста для n-грамм состоящих из трех символов

In [43]:
columns = ['Language', 'Family']
dftrain3 = pd.DataFrame(columns=columns) 
dftest3 = pd.DataFrame(columns=columns) 

In [44]:
for cat in train:
    category=train[cat]
    for f in category:
        dftrain3.loc[len(dftrain3)]=[cat, f] 
dftrain3.head()

,Language,Family
0,Japanese,Sada
1,Japanese,Isoda
2,Japanese,Takaoka
3,Japanese,Shige
4,Japanese,Shikuk


In [45]:
for cat in train:
    category=train[cat]
    for f in category:
        dftest3.loc[len(dftest3)]=[cat, f] 
dftest3.head()

,Language,Family
0,Japanese,Sada
1,Japanese,Isoda
2,Japanese,Takaoka
3,Japanese,Shige
4,Japanese,Shikuk


In [46]:
n=3

Разбиваем слова на n-граммы, записываем все в словарь, а так же создаем список n-грамм

In [47]:
train_ng=dict()
names=set()
for cat in tqdm(train):
    category=train[cat]
    cat_ng=dict()
    for f in category:
        ngs=[ng for ng in nltk.ngrams(f, n)]
        cat_ng[f]=ngs
        for ng in ngs:
            names.add(ng)
    train_ng[cat]=cat_ng

100%|██████████| 18/18 [00:00<00:00, 142.27it/s]


In [48]:
test_ng=dict()
for cat in tqdm(test):
    category=test[cat]
    cat_ng=dict()
    for f in category:
        ngs=[ng for ng in nltk.ngrams(f, n)]
        cat_ng[f]=ngs
        for ng in ngs:
            names.add(ng)
    test_ng[cat]=cat_ng

In [49]:
#Заполняем нулями обучающую матрицу
for name in tqdm(names):
    dftrain3[name]=dftrain3.fillna(0, inplace=True)

In [50]:
#Заполняем нулями тестовую матрицу
for name in tqdm(names):
    dftest3[name]=dftest3.fillna(0, inplace=True)

100%|██████████| 8221/8221 [1:19:03<00:00,  1.73it/s]


In [51]:
%time
#Заполняем 1 поля на пересечении фамилии и входящей в ее состав n-граммы в обучающей выборке
for cat in tqdm(all_categories):
    family = train_ng[cat]
    for f in family:
        ngs=family[f]
        for ng in ngs:
            dftrain3.loc[(dftrain3['Language'] == cat) & (dftrain3['Family'] == family), ng] = 1

  0%|          | 0/18 [00:00<?, ?it/s]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


100%|██████████| 18/18 [06:57<00:00, 23.22s/it]


In [52]:
#Заполняем 1 поля на пересечении фамилии и входящей в ее состав n-граммы в тестовой выборке
for cat in tqdm(all_categories):
    family = test_ng[cat]
    for f in family:
        ngs=family[f]
        for ng in ngs:
            dftest3.loc[(dftest3['Language'] == cat) & (dftest3['Family'] == family), ng] = 1

100%|██████████| 18/18 [01:52<00:00,  6.28s/it]


### Отделяем целевую переменную

In [53]:
Y_train=dftrain3['Language']
Y_test=dftest3['Language']

In [54]:
dftrain3.head()

,Language,Family,"(A, l, p)","(H, i, c)","(l, b, y)","(r, d, v)","(t, r, i)","(e, o, g)","(l, f, e)","(d, e, l)",...,"(v, g, o)","(T, a, v)","(P, e, d)","(k, n, e)","(N, a, n)","(V, o, u)","(N, u, t)","(a, s, a)","(H, ü, b)","(a, v, n)"
0,Japanese,Sada,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
1,Japanese,Isoda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
2,Japanese,Takaoka,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
3,Japanese,Shige,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
4,Japanese,Shikuk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None


In [55]:
dftrain3.drop('Language', axis=1, inplace=True)
dftest3.drop('Language', axis=1, inplace=True)
dftrain3.drop('Family', axis=1, inplace=True)
dftest3.drop('Family', axis=1, inplace=True)
dftrain3.drop(('a', 'v', 'n'), axis=1, inplace=True)
dftest3.drop(('a', 'v', 'n'), axis=1, inplace=True)

### Logistic Regression

In [56]:
dftest3.head()

,"(A, l, p)","(H, i, c)","(l, b, y)","(r, d, v)","(t, r, i)","(e, o, g)","(l, f, e)","(d, e, l)","(L, a, g)","(i, s, t)",...,"(v, r, y)","(v, g, o)","(T, a, v)","(P, e, d)","(k, n, e)","(N, a, n)","(V, o, u)","(N, u, t)","(a, s, a)","(H, ü, b)"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
clf = LogisticRegression(C=1)
clf.fit(dftrain3, Y_train) 
y_pred_l = clf.predict(dftrain3)
y_t_pred_l = clf.predict(dftest3)

In [59]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_train, y_pred_l, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_train, y_pred_l)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [60]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_test, y_t_pred_l, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_test, y_t_pred_l)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Random Forest

In [61]:
clf_rf = RandomForestClassifier(n_estimators=200, max_depth=15, min_samples_leaf=20, max_features=0.8, n_jobs=-1)
clf_rf.fit(dftrain3, Y_train)    

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features=0.8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [62]:
y_pred = clf_rf.predict(dftrain3)
y_t_pred = clf_rf.predict(dftest3)

In [63]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_train, y_pred, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_train, y_pred)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [64]:
print("F1-measure: {0:6.3f}".format(f1_score(Y_test, y_t_pred, average='macro')))
print("Accuracy: {0:6.3f}".format(accuracy_score(Y_test, y_t_pred)))

F1-measure:  0.035
Accuracy:  0.469


/home/alexey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


##  Часть 3. Нейронная сеть [35 баллов]


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс фамилии. 

Представление имени для классификации в этом случае: 
1 вариант: бинарная матрица размера (количество букв в корпусе $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$. То есть, используем one hot encoding для векторизации букв.  
2 вариант: Embedding'и символов. 

Выберите тот вариант, который вам проще или интереснее реализовать :) 

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами по accuracy, micro- и macro- F-measure, precision, recall. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor

##  Часть 4. Генерация фамилии [35 баллов]

Используйте архитектуру нейронной сети из Части 3 для генерации имени. В этот момент можно забыть про разбиение коллекции на обучающее и тестовое множество. 

Рассмотрите два сценария генерация имени:
* обучение нейронной сети по всей коллекции
* обучение нейронной сети с обуславливанием на язык

Для обуславливания на язык нужно изменить векторное представление фамилии. До этого момента мы разбивали фамилию на отдельные символы и находили векторное представление каждого символа. Теперь добавим в начало фамилии метку языка и будем ее считать первым символов фамилии. Так нейронная сеть научиться понимать, на каком языке написана фамилия. 

Пример: ```[rus bos i v a n o v eos pad pad pad]```

Когда будем генерировать новую фамилию, будем начинать процесс генерации не с символа начала последовательности ```bos```, а с символа языка и символа начала последовательности ```rus bos```. 

Привидите несколько примеров удачно сгенерированных фамилий. 